# DSSA
## Robert Tarasevič TSF–17

## 1 Labaratorinis darbas
### Tikslusis Fišerio testas

Dviejų požymių nepriklausomumo hipotezei patikrinti paprastai naudojamas chi-kvadrat testas. Žinoma, kad kriterijaus statistikos pasiskirstymas aproksimuojamas chi-kvadrat pasiskirstymu, todėl šiuo atveju p-reikšmės yra apytikslės.

Jeigu stebimi požymiai yra dvireikšmiai ir iš jų sudaroma 2 × 2 dydžio dažnių lentelė, tai nepriklausomumo hipotezei patikrinti galima naudoti tikslųjį Fišerio testą. R šiam tikslui turi funkciją fisher.test. Šiuo atveju p-reikšmė apskaičiuojama tiksliai.

UŽDUOTIS. Parašyti R funkciją, kuri hipotezę apie dviejų požymių nepriklausomumą 2 × 2 dydžio lentelėje patikrintų naudojant tikslųjį Fišerio testą.

Funkcijos argumentai: arba dažnių lentelė, arba kintamieji, iš kurios dažnių lentelė sudaroma funkcijos viduje. Jeigu reikia, galima perduoti reikšmingumo lygmens α reikšmę.

Funkcija turi grąžinti p-reikšmę. Taip pat gali būti išvedama papildoma informacija, pavyzdžių sprendimas apie hipotezės priėmimą arba atmetimą, jeigu funkcijai nurodomas reikšmingumo lygmuo α.

Funkcijos veikimą galima išbandyti su duomenimis iš lentelės ksimpah.csv.
***

Kadangi šiuo atvėju yra tik 4 gardelės, tai tradiciškai jų reikšmės žymimos raidėmis $(\sigma_{11}=a, \sigma_{12} = b, \sigma_{2,1}=c, \sigma_{22}=d)$


| dažnių lentelė | $x_1$ | $x_2$ | $\Sigma$ |
| --- | --- | --- | ---
| $y_1$ | a | b | a+b |
| $y_2$ | c | d | c+d |
| $\Sigma$ | a+c | b+d | a+b+c+d |

Tarkime, kad lentelės eilučių sumos a+b, c+d ir stulpelių sumos a+c, b+d yra fiksuoti dydžiai. Tada, žinant vienos iš gardelių dažnį, žinomi ir likusieji. Tuomet tikimybė, kad tarp $n=a+b+c+d$ dvireikšmių kintamųjų poros (X,Y) stebėjimų reikšmių ($x_1, y_1$) pasirodys *a* kartų (kartu ($x_1, y_2$) - *c* kartų, ($x_2, y_1$) - *b* kartų, ($x_2, y_2$) - *d* kartų), užrašoma kaip hipergeometrinio skirstinio tikimybė:
$$P(a,b,c,d) = \frac{(a+b)!(c+d)!(a+c)!(b+d)!}{a!b!c!d!n!}$$

Apsibrėžkime hipotezę:

$H_0$ teigia, kad požymiai nepriklausomi, alternatyva $H_1$ - kad priklausomi
$$H_0: p_{ij} = p_ip_j,\quad \text{su visais i=1,2;}\; \text{j=1,2} \\ H_1: p_{ij} \ne p_ip_j,\quad \text{bent vienai porai (i,j)}$$

$H_0$ hipotezei patikrinti, reikia, pagal hipergeometrinio skirstinio tikimybės formulę apskaičiuoti tikimybes dažnių lentelei ir visoms galimoms lentelėms, kurių pirmosios gardelės dažnis mažesnis už *a*. Jeigu sudėjus gautas tikimybes **suma neviršija pusės pasirinkto reikšmingumo lygmens**, t.y. $\alpha/2$, nulinę hipotezę $H_0$ **reikia atmesti** (požymiai statistiškai reikšmingai priklausomi). Jeigu suma viršija $\alpha/2$ tai reikia skaičiuoti hipergeometrinio skirstinio tikimybes dažnių lentelei ir visoms galimoms, kurių pirmosios gardelės dažnis didesnis už *a*. Jeigu sudėjus gautas tikimybes **suma neviršija $\alpha/2$**, nulinę hipotezę $H_0$ **reikia atmesti**. Priešingu atvėju $H_0$ atmesti nėra pagrindo.

Sudarome dažnių lentelę:

In [1]:
a = 3
b = 16
c = 2
d = 21

dazniu_lentele <- rbind(
    c(a,b),
    c(c,d))

dazniu_lentele

3,16
2,21


Funkcija skaičiuojanti hipergeometrinio skirstinio tikimybę ir leidžianti perskaičiuoti dažnių lentelę pakeitus **a**:

In [2]:
P = function(dazniu_lentele, a=dazniu_lentele[1,1]) {
    if (dazniu_lentele[1,1]==a) {
        a=dazniu_lentele[1,1]
        b = dazniu_lentele[1,2]
        c = dazniu_lentele[2,1]
        d = dazniu_lentele[2,2]
    } else {
        #print("lentele pakeista!")
        a = a
        b = dazniu_lentele[1,1] + dazniu_lentele[1,2] - a
        c = dazniu_lentele[1,1] + dazniu_lentele[2,1] - a
        d = dazniu_lentele[2,2] - dazniu_lentele[1,1] + a
        
    }
    
    n = a+b+c+d
    result = prod(factorial(c(a+b, c+d, a+c, b+d))) / 
    prod(factorial(c(a,b,c,d,n)))

    return(result)
}

$H_0$ hipotezei patikrinti reikia paskaičiuoti visų galimų dažnių lentelių tikimybių sumą, kurių pirmosios gardelės dažnis yra mažesnis už **a**.

In [3]:
P(dazniu_lentele)

[1] 0.2881935

In [4]:
P_sum = function(dazniu_lentele, maziau=T) {
    if (maziau) {
        galimos_a = seq(from=0, to=dazniu_lentele[1,1], by=1)
    } else {
        A = min(dazniu_lentele[1,1]+dazniu_lentele[1,2], dazniu_lentele[1,1]+
                dazniu_lentele[2,1])
        galimos_a = seq(from=dazniu_lentele[1,1], to=A, by=1)
    }
     
    suma = 0
    for (i in galimos_a) {
        #print(P(dazniu_lentele, a=i))
        suma = suma + P(dazniu_lentele, a=i)
    }
    
    return(suma)
    }

In [5]:
P_sum(dazniu_lentele)

[1] 0.8815331

Patikriname gautą reikšmę su *R* funkcija *fisher.test*

In [6]:
P_sum(dazniu_lentele) == fisher.test(dazniu_lentele, alternative="less")$p.value
print(fisher.test(dazniu_lentele, alternative="less")$p.value)

[1] TRUE

[1] 0.8815331


Aukščiau minėjome, kad jeigu tikimybių suma neviršiją $\alpha / 2$,nulinę hipotezę $H_0$ reikia atmesti.

Patikrinkime tai.

In [7]:
alpha = 0.05

if (P_sum(dazniu_lentele) <= alpha/2) {
    print("Hipoteze H0 riekia atmesti.")
} else {
    print("Hipoteze H0 atmesti negalime.")
}

[1] "Hipoteze H0 atmesti negalime."


Suma viršija mūsų parinkta reikšmingumo lygmenį, todėl reikia skaičiuoti **P** tikimybes duotai dažnių lentelei ir visoms galimoms lentelėms, kurių pirmos gardelės dažnis yra didesnis už **a**, tam mum reikės apskaičiuoti kokios galimos reikšmės didesnės **a**, kad lentelės suma nepasikeistų.

In [8]:
A = min(a+b, a+c)
A

[1] 5

Analogiškai apskaičoiuosime P tikimybęs duotai dažnių lentelei ir visoms galimoms lentelėms su pirmos gardelės reikšmėmis didesnėmis už **a** reikšmę ir sudėsim gautas tikimybes

In [9]:
P_sum(dazniu_lentele, maziau=F)

[1] 0.4066604

Patikrinsime ar gauta reikšmė sutampa su *R* programos skaičiavimais naudojant funkciją *fisher.test*

In [10]:
fish = fisher.test(dazniu_lentele, alternative="greater")$p.value
P_sum(dazniu_lentele, maziau=F) == fish
print(fish)

[1] TRUE

[1] 0.4066604


Jeigu sudėtų tikimybių suma neviršija pusės pasirinkto reikšmingumo lygmens, tai nulinę hipotezę $H_0$ reikia atmesti.

In [11]:
alpha = 0.05

if (P_sum(dazniu_lentele, maziau=F) <= alpha/2) {
    print("Hipoteze H0 riekia atmesti.")
} else {
    print("Hipoteze H0 atmesti negalime.")
}

[1] "Hipoteze H0 atmesti negalime."


#### IŠVADOS:

Hipotezė $H_0: p_{ij} = p_ip_j,\quad \text{su visais i=1,2;}\; \text{j=1,2}\;$ **priimame**, o tai reiškia, kad požymiai yra nepriklausomi.
***
Pilna R programa:

In [12]:
H0 = function(dazniu_lentele, alpha=0.05) {
    
    tmp_res = P_sum(dazniu_lentele)
    
    if (tmp_res <= alpha/2) {
        cat("Hipoteze H0 riekia atmesti.", '\n','p1: \t', tmp_res)
    } else {
        cat("Hipoteze H0 atmesti negalime. Tikriname toliau. \n")
        
        tmp_res2 = P_sum(dazniu_lentele, maziau=F)
        if (P_sum(dazniu_lentele, maziau=F) <= alpha/2) {
            cat("\nHipoteze H0 riekia atmesti. pozymiai yra priklausomi.", 
               '\n','p1: \t', tmp_res, '\n','p2: \t', tmp_res2)
        } else {
            cat("Hipoteze H0 atmesti negalime. Pozymiai yra nepriklausomi.", 
                '\n','p1: \t', tmp_res, '\n','p2: \t', tmp_res2)
        }

        
        
    }
    
}

In [13]:
H0(dazniu_lentele, alpha=0.05)

Hipoteze H0 atmesti negalime. Tikriname toliau. 
Hipoteze H0 atmesti negalime. Pozymiai yra nepriklausomi. 
 p1: 	 0.8815331 
 p2: 	 0.4066604